# DEPENDENCY INSTALLATION    

In [ ]:
!pip install -q transformers==4.45.0
!pip install -q datasets==2.16.1
!pip install -q accelerate==1.0.0
!pip install -q evaluate==0.4.3
!pip install -q jiwer==3.0.3
!pip install -q tensorboard soundfile librosa
!pip install -q "bitsandbytes>=0.43.0"
!pip install -q num2words bangla huggingface_hub

print("✅ All dependencies installed.")

# ENVIRONMENT SETUP & CORE IMPORTS  

In [ ]:
import os, gc, random, glob, re, warnings, logging
import numpy as np
import pandas as pd
import torch
import librosa
from dataclasses import dataclass
from typing import Any, Dict, List, Union

warnings.filterwarnings("ignore")

os.environ["CUDA_VISIBLE_DEVICES"]    = "0"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TRANSFORMERS_VERBOSITY"]  = "warning"

from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    TrainerCallback,
)
import evaluate
from datasets import Dataset, DatasetDict, Audio
from huggingface_hub import HfApi, login, snapshot_download


import torch.serialization
try:
    torch.serialization.add_safe_globals(
        [np.ndarray, np.dtype, np.core.multiarray._reconstruct]
    )
except AttributeError:
    try:
        torch.serialization.add_safe_globals(
            [np.ndarray, np.dtype, np._core.multiarray._reconstruct]
        )
    except Exception as e:
        print(f"⚠️  numpy safe globals registration skipped: {e}")

_orig_load = torch.load
def _patched_load(*args, **kwargs):
    kwargs.setdefault("weights_only", False)
    return _orig_load(*args, **kwargs)
torch.load = _patched_load

logging.basicConfig(
    format  = "%(asctime)s | %(levelname)s | %(message)s",
    datefmt = "%H:%M:%S",
    level   = logging.WARNING,
)

# ── Device Report ─────────────────────────────────────────────────────────────
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    vram = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🖥️  GPU  : {torch.cuda.get_device_name(0)}  |  VRAM : {vram:.1f} GB")
    print(f"   Visible GPU count : {torch.cuda.device_count()}  (must be 1)")
    torch.cuda.empty_cache()
else:
    print("⚠️  Running on CPU — training will be very slow.")

gc.collect()
print("✅ Environment ready.")

# PROJECT CONFIGURATION

- Edit all paths and hyperparameters here before running

In [ ]:
# ── Model & HuggingFace Hub ───────────────────────────────────────────────────
MODEL_NAME     = "bengaliAI/tugstugi_bengaliai-regional-asr_whisper-medium"
HF_TOKEN       = ""
HF_OUTPUT_REPO = "bitwisemind/sam_15000_clean_text_full_model"
UPLOAD_CHECKPOINTS = True   # Set False to skip HuggingFace uploads

# ── Dataset Paths ─────────────────────────────────────────────────────────────
METADATA_PATH   = "/kaggle/input/datasets/sazzadhossainsazzad/1-to-121-audio-25s-zip/annotation.csv"
AUDIO_BASE_PATH = "/kaggle/input/datasets/sazzadhossainsazzad/1-to-121-audio-25s-zip/"
DATA_START      = 1
DATA_END        = 15229
TRAIN_SPLIT     = 0.9     # 90% train / 10% validation
OUTPUT_PATH     = "./whisper-fullweight-output"

# ── Training Hyperparameters ──────────────────────────────────────────────────

PER_DEVICE_TRAIN_BATCH = 8   # Per-GPU batch size
GRAD_ACCUM_STEPS       = 2   # Effective batch = 8 × 2 = 16
PER_DEVICE_EVAL_BATCH  = 8
LEARNING_RATE          = 5e-6
WARMUP_STEPS           = 100
TARGET_EPOCHS          = 6

# ── Audio / Text Constraints ──────────────────────────────────────────────────
MAX_AUDIO_DURATION = 30
MAX_TEXT_LENGTH    = 1000

# ── Data Augmentation ─────────────────────────────────────────────────────────
USE_AUGMENTATION   = True
AUGMENTATION_PROB  = 0.3    # Probability of augmenting a training sample

print("✅ Configuration loaded.")
print(f"   Model            : {MODEL_NAME}")
print(f"   Data range       : rows {DATA_START}–{DATA_END}")
print(f"   Effective batch  : {PER_DEVICE_TRAIN_BATCH * GRAD_ACCUM_STEPS}")
print(f"   Learning rate    : {LEARNING_RATE}  |  Epochs : {TARGET_EPOCHS}")



# HUGGINGFACE LOGIN & COMPATIBILITY SHIM    

In [ ]:
import huggingface_hub.hf_api as _hf_api_module

class _HfFolderShim:
    """Minimal replacement for the deprecated huggingface_hub.HfFolder."""
    _token = HF_TOKEN

    @classmethod
    def get_token(cls):   return cls._token
    @classmethod
    def save_token(cls, token): cls._token = token
    @classmethod
    def delete_token(cls): cls._token = None

if not hasattr(_hf_api_module, "HfFolder"):
    _hf_api_module.HfFolder = _HfFolderShim

# ── Authenticate ──────────────────────────────────────────────────────────────
if HF_TOKEN:
    login(token=HF_TOKEN, add_to_git_credential=False)
    print("✅ HuggingFace login successful.")
else:
    print("⚠️  No HF_TOKEN provided — uploads will be skipped.")


# CHECKPOINT DISCOVERY (LOCAL & REMOTE)

- Downloads an existing checkpoint from HF Hub if one is present, then scans the local output directory for the latest step.

In [ ]:
def download_checkpoint_from_hf(repo_id: str, local_dir: str, token: str) -> None:
    """Pull the latest checkpoint from a HuggingFace dataset repo."""
    if os.path.exists(local_dir) and os.listdir(local_dir):
        print(f"ℹ️  {local_dir} already populated — skipping HF download.")
        return
    print(f"📥 Downloading checkpoint from HuggingFace: {repo_id}")
    try:
        snapshot_download(
            repo_id=repo_id, repo_type="dataset",
            local_dir=local_dir, token=token,
        )
        print(f"✅ Checkpoint downloaded → {local_dir}")
    except Exception as e:
        print(f"ℹ️  No remote checkpoint found (first run or empty repo): {e}")


def find_latest_checkpoint(output_dir: str):
    """Return the path to the highest-numbered checkpoint folder, or None."""
    if not os.path.exists(output_dir):
        return None
    folders = glob.glob(os.path.join(output_dir, "checkpoint-*"))
    candidates = []
    for f in folders:
        m = re.search(r"checkpoint-(\d+)", f)
        if m:
            candidates.append((int(m.group(1)), f))
    return max(candidates, key=lambda x: x[0])[1] if candidates else None


download_checkpoint_from_hf(HF_OUTPUT_REPO, OUTPUT_PATH, HF_TOKEN)

RESUME_FROM_CHECKPOINT = find_latest_checkpoint(OUTPUT_PATH)

if RESUME_FROM_CHECKPOINT:
    print(f"✅ Resuming from checkpoint : {RESUME_FROM_CHECKPOINT}")
else:
    print("ℹ️  No checkpoint found — training will start from scratch.")



# MODEL & PROCESSOR LOADING

- Loads the Whisper model in float32 (no quantization).Gradient checkpointing is enabled to reduce VRAM usage.

In [ ]:
torch.cuda.empty_cache(); gc.collect()

print(f"📦 Loading model : {MODEL_NAME}")
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)

# Disable KV cache (incompatible with gradient checkpointing during training)
model.config.use_cache = False
model.gradient_checkpointing_enable()

# Ensure all parameters are trainable (full fine-tuning, no freezing)
for param in model.parameters():
    param.requires_grad = True

# ── Parameter Summary ─────────────────────────────────────────────────────────
total_params     = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"   Total parameters     : {total_params:,}")
print(f"   Trainable parameters : {trainable_params:,}  ({100 * trainable_params / total_params:.1f}%)")

if device == "cuda":
    used  = torch.cuda.memory_allocated() / 1024**3
    total = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"   VRAM after loading   : {used:.2f} / {total:.2f} GB  ({total - used:.2f} GB free)")

# ── Processor ─────────────────────────────────────────────────────────────────
feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_NAME)
tokenizer  = WhisperTokenizer.from_pretrained(MODEL_NAME, language="Bengali", task="transcribe")
processor  = WhisperProcessor.from_pretrained(MODEL_NAME, language="Bengali", task="transcribe")

print("✅ Model and processor loaded.")

# TEXT NORMALIZATION & CLEANING UTILITIES

In [ ]:
from num2words import num2words

# ── Number-to-Words Conversion ────────────────────────────────────────────────

def _to_bangla_words(n: int, is_year: bool = False) -> str:
    """Convert an integer to its Bangla word representation."""
    try:
        return num2words(n, lang="bn", to="year") if is_year else num2words(n, lang="bn")
    except Exception:
        return num2words(n, lang="bn")


def convert_numbers_to_bangla(text: str) -> str:
    """Replace all English digit sequences in text with Bangla words.

    4-digit numbers in the range 1000–2099 are treated as calendar years.
    """
    def _replace(m):
        token = m.group(0)
        n = int(token)
        return _to_bangla_words(n, is_year=(len(token) == 4 and 1000 <= n <= 2099))
    return re.sub(r"\d+", _replace, text)


def normalize_text(text: str) -> str:
    """Normalize a transcript: strip whitespace and expand digits to Bangla words."""
    if pd.isna(text) or not text:
        return ""
    text = " ".join(str(text).strip().split())
    return convert_numbers_to_bangla(text)


# ── Bangla Character Filtering ────────────────────────────────────────────────

def _is_bangla(char: str) -> bool:
    return 0x0980 <= ord(char) <= 0x09FF


def _is_allowed(char: str) -> bool:
    """Return True if the character is permitted in a clean Bengali transcript."""
    return (
        _is_bangla(char)
        or char in " \n\r\t"
        or char.isdigit()
        or char in "।.,?!-—:;\"'()[]{}"
    )


def clean_annotation_text(text: str):
    """Remove non-Bengali / non-allowed characters from a transcript string.

    Returns:
        cleaned_text (str), chars_removed (int), had_garbage (bool)
    """
    cleaned = [c for c in text if _is_allowed(c)]
    removed = len(text) - len(cleaned)
    return "".join(cleaned), removed, removed > 0


print("✅ Text normalization utilities ready.")

# AUDIO AUGMENTATION PIPELINE

- Applies realistic degradations to a random subset of each audio clip to improve model robustness on noisy, real-world speech.  

In [ ]:
from scipy import signal
from scipy.signal import butter, lfilter

# ── Individual Augmentation Functions ────────────────────────────────────────

def bandpass_filter(audio: np.ndarray, sr: int,
                    lowcut: int = 300, highcut: int = 3400) -> np.ndarray:
    """Simulate telephone-quality audio by bandpass filtering."""
    nyq  = sr / 2
    low  = max(0.001, min(lowcut  / nyq, 0.999))
    high = max(0.001, min(highcut / nyq, 0.999))
    if low >= high:
        return audio
    b, a = butter(4, [low, high], btype="band")
    return lfilter(b, a, audio).astype(np.float32)


def build_rir(sr: int, room_size: str = "medium") -> np.ndarray:
    """Construct a synthetic Room Impulse Response for convolution reverb."""
    configs = {
        "small":  {"rt60": 0.15, "n_ref": 6,  "max_delay": 0.05},
        "medium": {"rt60": 0.40, "n_ref": 10, "max_delay": 0.12},
        "large":  {"rt60": 0.90, "n_ref": 16, "max_delay": 0.25},
    }
    cfg     = configs[room_size]
    rt60    = cfg["rt60"]
    rir_len = int(rt60 * sr * 2)
    rir     = np.zeros(rir_len)

    for _ in range(cfg["n_ref"]):
        delay = int(random.uniform(0.002, cfg["max_delay"]) * sr)
        if delay < rir_len:
            rir[delay] += random.uniform(0.3, 0.9)

    tail_start = int(0.02 * sr)
    tail = (np.exp(-np.linspace(0, 6.9 / rt60, rir_len - tail_start))
            * np.random.randn(rir_len - tail_start) * 0.5)
    rir[tail_start:] += tail
    mx = np.abs(rir).max()
    return (rir / mx if mx > 0 else rir).astype(np.float32)


def apply_reverb(segment: np.ndarray, sr: int, room_size: str = None) -> np.ndarray:
    """Convolve a segment with a synthetic room impulse response."""
    if room_size is None:
        room_size = random.choice(["small", "medium", "large"])
    rir = build_rir(sr, room_size)
    wet = signal.fftconvolve(segment, rir, mode="full")[:len(segment)]
    mx  = np.abs(wet).max()
    if mx > 0:
        wet /= mx
    mix = random.uniform(0.4, 0.85)
    return ((1 - mix) * segment + mix * wet).astype(np.float32)


def apply_echo(segment: np.ndarray, sr: int) -> np.ndarray:
    """Add multi-tap echo at random delay and decay settings."""
    result        = segment.copy()
    n_taps        = random.randint(2, 4)
    base_delay_ms = random.randint(150, 800)
    decay         = random.uniform(0.4, 0.75)
    for tap in range(1, n_taps + 1):
        delay_samples = int((base_delay_ms * tap) * sr / 1000)
        if delay_samples < len(result):
            echo = np.zeros_like(result)
            echo[delay_samples:] = result[:-delay_samples] * (decay ** tap)
            result = result + echo
    return result.astype(np.float32)


def apply_noise(segment: np.ndarray, sr: int) -> np.ndarray:
    """Mix pink and white noise into the segment at a low SNR."""
    white = np.random.randn(len(segment))
    fft   = np.fft.rfft(white)
    freqs = np.fft.rfftfreq(len(white)); freqs[0] = 1e-6
    pink  = np.fft.irfft(fft / np.sqrt(freqs), n=len(white))
    mx    = np.abs(pink).max()
    if mx > 0:
        pink /= mx
    noise_factor = random.uniform(0.02, 0.08)
    mix          = random.uniform(0.3, 0.7)
    return (segment + noise_factor * (mix * white + (1 - mix) * pink)).astype(np.float32)


def apply_clipping(segment: np.ndarray, threshold: float = None) -> np.ndarray:
    """Simulate ADC clipping distortion."""
    if threshold is None:
        threshold = random.uniform(0.4, 0.75)
    return np.clip(segment, -threshold, threshold).astype(np.float32)


# ── Main Augmentation Orchestrator ────────────────────────────────────────────

def augment_audio_segments(audio_array: np.ndarray, sr: int = 16000,
                            augment_percentage: float = 0.30) -> np.ndarray:
    """Apply random augmentations to non-overlapping segments of an audio clip.

    Augments up to `augment_percentage` of the total duration by selecting
    random 3–6 second windows and applying a stochastic chain of effects
    (noise, echo, reverb, clipping, pitch shift, time stretch).
    """
    augmented     = audio_array.copy().astype(np.float32)
    aug_budget    = len(augmented) / sr * augment_percentage
    used_segments = []
    total_aug_time = 0.0

    while total_aug_time < aug_budget:
        seg_dur     = min(random.uniform(3, 6), aug_budget - total_aug_time)
        seg_samples = int(seg_dur * sr)
        if seg_samples <= 0:
            break

        # Find a non-overlapping start position
        start = None
        for _ in range(60):
            if len(augmented) <= seg_samples:
                break
            candidate = random.randint(0, len(augmented) - seg_samples)
            end       = candidate + seg_samples
            if all(end <= s or candidate >= e for s, e in used_segments):
                start = candidate
                break

        if start is None:
            break

        end = start + seg_samples
        used_segments.append((start, end))
        seg = augmented[start:end].copy()

        # ── Stochastic effect chain ────────────────────────────────────────
        if random.random() < 0.65: seg = apply_noise(seg, sr)
        if random.random() < 0.55: seg = apply_echo(seg, sr)
        if random.random() < 0.60: seg = apply_reverb(seg, sr)
        if random.random() < 0.30: seg = apply_clipping(seg)
        if random.random() < 0.20: seg = bandpass_filter(seg, sr)
        if random.random() < 0.25:
            seg = librosa.effects.pitch_shift(seg, sr=sr, n_steps=random.uniform(-3, 3))
        if random.random() < 0.25:
            rate = random.uniform(0.80, 1.20)
            seg  = librosa.effects.time_stretch(seg, rate=rate)
            seg  = seg[:seg_samples] if len(seg) > seg_samples else np.pad(seg, (0, seg_samples - len(seg)))

        # Normalize segment peak
        mx = np.abs(seg).max()
        if mx > 0:
            seg = seg / mx * 0.9

        augmented[start:end] = seg.astype(np.float32)
        total_aug_time += seg_dur

    return augmented.astype(np.float32)


print("✅ Audio augmentation pipeline ready.")

# DATA CLEANING: GARBAGE TEXT REMOVAL

- Strips non-Bengali characters from all transcript annotations before the dataset is built, improving label quality.

In [ ]:

print("🧹 Cleaning annotation texts...")

df_raw      = pd.read_csv(METADATA_PATH)
df_filtered = df_raw.iloc[DATA_START - 1 : DATA_END].copy()

# Auto-detect the transcription column by common keyword patterns
transcription_col = next(
    (c for c in df_filtered.columns
     if any(k in c.lower() for k in ("gt", "text", "annotation", "transcription"))),
    None,
)

total_cleaned, total_removed = 0, 0

if transcription_col:
    for idx, row in df_filtered.iterrows():
        raw = row[transcription_col]
        if pd.notna(raw):
            cleaned, removed, had_garbage = clean_annotation_text(str(raw))
            if had_garbage:
                df_filtered.at[idx, transcription_col] = cleaned
                total_cleaned += 1
                total_removed += removed

    print(f"   Rows scanned   : {len(df_filtered)}")
    print(f"   Rows cleaned   : {total_cleaned}")
    print(f"   Chars removed  : {total_removed}")
else:
    print("⚠️  Transcription column not detected — skipping garbage removal.")

df_metadata_cleaned = df_filtered.copy()
print("✅ Annotation cleaning complete.")

# DATASET CONSTRUCTION

- Validates audio files, normalizes transcripts, shuffles, and splits into train / validation sets.

In [ ]:
def load_dataset_from_csv(
    metadata_path: str,
    audio_base_path: str,
    start_row: int = 1,
    end_row: int   = 5000,
    split_ratio: float = 0.9,
    max_text_length: int = 1000,
) -> DatasetDict:
    """Build a HuggingFace DatasetDict from a CSV metadata file.

    Each row must contain an audio file path and a transcript.
    Invalid paths, empty texts, and over-length texts are silently dropped.
    """
    df = df_metadata_cleaned.copy() if "df_metadata_cleaned" in globals() \
         else pd.read_csv(metadata_path).iloc[start_row - 1 : end_row].copy()

    cols = df.columns.tolist()
    # Resolve column names for audio path and transcript
    if   "audio_path" in cols and "gt"            in cols: path_col, text_col = "audio_path", "gt"
    elif "path"       in cols and "text"          in cols: path_col, text_col = "path",       "text"
    elif "audio_path" in cols and "transcription" in cols: path_col, text_col = "audio_path", "transcription"
    else:                                                   path_col, text_col = cols[0],       cols[1]

    valid, skipped = [], 0
    for _, row in df.iterrows():
        try:
            # Resolve absolute audio path
            ap = str(row[path_col]).strip()
            if   ap.startswith("./"): ap = os.path.join(audio_base_path, ap[2:])
            elif not ap.startswith("/"): ap = os.path.join(audio_base_path, ap)

            text = normalize_text(row[text_col])

            # Quality gates
            if not text or len(text) < 3 or len(text) > max_text_length:
                skipped += 1; continue
            if not os.path.exists(ap) or os.path.getsize(ap) == 0:
                skipped += 1; continue

            valid.append({"path": ap, "sentence": text})
        except Exception:
            skipped += 1

    if not valid:
        raise ValueError("No valid samples found. Check METADATA_PATH and AUDIO_BASE_PATH.")

    random.shuffle(valid)
    split      = max(1, int(len(valid) * split_ratio))
    train_data = valid[:split]
    val_data   = valid[split:]

    def to_hf_dataset(data):
        ds = Dataset.from_dict({
            "audio":    [x["path"]     for x in data],
            "sentence": [x["sentence"] for x in data],
        })
        return ds.cast_column("audio", Audio(sampling_rate=16000))

    print(f"   Valid samples  : {len(valid)}  (skipped {skipped})")
    print(f"   Train / Val    : {len(train_data)} / {len(val_data)}")
    return DatasetDict({"train": to_hf_dataset(train_data), "test": to_hf_dataset(val_data)})


print("📂 Building dataset...")
dataset = load_dataset_from_csv(
    METADATA_PATH, AUDIO_BASE_PATH,
    start_row       = DATA_START,
    end_row         = DATA_END,
    split_ratio     = TRAIN_SPLIT,
    max_text_length = MAX_TEXT_LENGTH,
)
print("✅ Dataset ready.")

# FEATURE EXTRACTION & DATASET PREPROCESSING

- Converts raw audio arrays to log-mel spectrograms and tokenizes transcripts. Augmentation is applied only to train.

In [ ]:
def prepare_dataset(batch: dict, augment: bool = False) -> dict:
    """Map function: audio waveform + transcript → model input features."""
    audio = batch["audio"]
    arr   = audio["array"].astype(np.float32)
    sr    = audio["sampling_rate"]

    # Resample to 16 kHz if necessary
    if sr != 16000:
        arr = librosa.resample(arr, orig_sr=sr, target_sr=16000)
        sr  = 16000

    # Clip to max duration (Whisper supports up to 30 s)
    max_samples = int(MAX_AUDIO_DURATION * sr)
    if len(arr) > max_samples:
        arr = arr[:max_samples]

    # Stochastic augmentation (training only)
    if augment and USE_AUGMENTATION and random.random() < AUGMENTATION_PROB:
        arr = augment_audio_segments(arr, sr=sr)

    batch["input_features"] = feature_extractor(arr, sampling_rate=sr).input_features[0]
    batch["labels"]         = tokenizer(batch["sentence"], max_length=448, truncation=True).input_ids
    return batch


print("⚙️  Preprocessing training set  (augmentation ON)...")
dataset["train"] = dataset["train"].map(
    lambda x: prepare_dataset(x, augment=True),
    remove_columns=["audio", "sentence"],
    num_proc=1,
)

print("⚙️  Preprocessing validation set (augmentation OFF)...")
dataset["test"] = dataset["test"].map(
    lambda x: prepare_dataset(x, augment=False),
    remove_columns=["audio", "sentence"],
    num_proc=1,
)

print("✅ Feature extraction complete.")

# DATA COLLATOR & EVALUATION METRIC

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """Pad input features and label token sequences to the same length within a batch.

    Labels are padded with -100 so that padding tokens are ignored by the loss.
    The decoder start token is stripped from labels when it appears as the first token
    (the model adds it automatically during forward pass).
    """
    processor: Any
    decoder_start_token_id: int

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # Pad audio features
        input_features = [{"input_features": f["input_features"]} for f in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Pad label sequences and mask padding with -100
        label_features = [{"input_ids": f["labels"]} for f in features]
        labels_batch   = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )
        # Remove decoder start token if prepended (avoids double-counting)
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch


data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

# ── WER Metric ────────────────────────────────────────────────────────────────
wer_metric = evaluate.load("wer")

def compute_metrics(pred) -> Dict[str, float]:
    """Decode predictions and references, then compute Word Error Rate."""
    pred_ids  = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    pred_str  = tokenizer.batch_decode(pred_ids,  skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    return {"wer": 100 * wer_metric.compute(predictions=pred_str, references=label_str)}


print("✅ Data collator and WER metric ready.")

# HUGGINGFACE UPLOAD UTILITIES & CALLBACK

- Checkpoints are automatically pushed to the HF Hub after each save_steps interval.

In [ ]:
def upload_to_hf(local_path: str, repo_id: str, commit_msg: str = None) -> None:
    """Upload a local folder to a HuggingFace dataset repository."""
    if not UPLOAD_CHECKPOINTS or not HF_TOKEN:
        return
    try:
        name = os.path.basename(local_path)
        HfApi().upload_folder(
            folder_path    = local_path,
            repo_id        = repo_id,
            repo_type      = "dataset",
            path_in_repo   = name,
            commit_message = commit_msg or f"Upload {name}",
            token          = HF_TOKEN,
        )
        print(f"   📤 Uploaded → {repo_id}/{name}")
    except Exception as e:
        print(f"   ⚠️  Upload failed (non-fatal): {e}")


class HFUploadCallback(TrainerCallback):
    """Trainer callback that pushes every saved checkpoint to HuggingFace Hub."""

    def __init__(self, repo_id: str):
        self.repo_id = repo_id

    def on_save(self, args, state, control, **kwargs):
        if state.global_step > 0:
            ckpt_path = os.path.join(args.output_dir, f"checkpoint-{state.global_step}")
            if os.path.exists(ckpt_path):
                upload_to_hf(
                    ckpt_path, self.repo_id,
                    commit_msg=f"Checkpoint — step {state.global_step}",
                )


print("✅ Upload callback registered.")

# TRAINING ARGUMENTS- Optimizer

- Optimizer  : adamw_bnb_8bit (saves ~2.4 GB VRAM vs fp32 AdamW)



- Precision  : fp16 mixed precision                  


 - Scheduler  : cosine with warmup

In [ ]:
train_samples   = len(dataset["train"])
steps_per_epoch = max(1, train_samples // (PER_DEVICE_TRAIN_BATCH * GRAD_ACCUM_STEPS))
total_steps     = steps_per_epoch * TARGET_EPOCHS
eval_save_steps = 6000

training_args = Seq2SeqTrainingArguments(
    output_dir = OUTPUT_PATH,

    # ── Batching ──────────────────────────────────────────────────────────────
    per_device_train_batch_size = PER_DEVICE_TRAIN_BATCH,
    gradient_accumulation_steps = GRAD_ACCUM_STEPS,
    per_device_eval_batch_size  = PER_DEVICE_EVAL_BATCH,

    # ── Optimizer & Scheduler ─────────────────────────────────────────────────
    optim             = "adamw_bnb_8bit",   # 8-bit moments → significant VRAM savings
    learning_rate     = LEARNING_RATE,
    warmup_steps      = WARMUP_STEPS,
    lr_scheduler_type = "cosine",

    # ── Duration ──────────────────────────────────────────────────────────────
    max_steps = total_steps,

    # ── Memory ────────────────────────────────────────────────────────────────
    gradient_checkpointing = True,
    fp16                   = True,
    torch_compile          = False,

    # ── Logging ───────────────────────────────────────────────────────────────
    disable_tqdm       = False,
    logging_first_step = True,
    logging_steps      = 25,
    log_level          = "warning",

    # ── Evaluation & Checkpointing ────────────────────────────────────────────
    evaluation_strategy = "steps",
    eval_steps          = eval_save_steps,
    save_steps          = eval_save_steps,
    save_total_limit    = 2,           # Keep only the 2 most recent checkpoints

    # ── Generation ────────────────────────────────────────────────────────────
    predict_with_generate = True,
    generation_max_length = 225,

    # ── Best-Model Tracking ───────────────────────────────────────────────────
    load_best_model_at_end = True,
    metric_for_best_model  = "wer",
    greater_is_better      = False,    # Lower WER is better

    # ── Miscellaneous ─────────────────────────────────────────────────────────
    resume_from_checkpoint = RESUME_FROM_CHECKPOINT,
    report_to              = ["tensorboard"],
    remove_unused_columns  = False,
    label_names            = ["labels"],
    dataloader_num_workers = 0,
)

print("✅ Training arguments configured.")
print(f"   Train samples : {train_samples}  |  Steps/epoch : {steps_per_epoch}")
print(f"   Total steps   : {total_steps}  |  Eval/save every : {eval_save_steps} steps")
print(f"   Optimizer     : adamw_bnb_8bit  |  Precision : fp16")

# TRAINER INITIALIZATION

In [ ]:
os.makedirs(OUTPUT_PATH, exist_ok=True)
processor.save_pretrained(OUTPUT_PATH)   # Save processor alongside model artifacts

callbacks = [HFUploadCallback(repo_id=HF_OUTPUT_REPO)] if UPLOAD_CHECKPOINTS else []

trainer = Seq2SeqTrainer(
    args            = training_args,
    model           = model,
    train_dataset   = dataset["train"],
    eval_dataset    = dataset["test"],
    data_collator   = data_collator,
    compute_metrics = compute_metrics,
    tokenizer       = processor.feature_extractor,
    callbacks       = callbacks,
)

if device == "cuda":
    used  = torch.cuda.memory_allocated() / 1024**3
    total = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🖥️  VRAM before training : {used:.2f} / {total:.2f} GB")

print("✅ Trainer ready.")

# TRAINING EXECUTION

In [ ]:
print("\n" + "═" * 65)
print("🚀  STARTING TRAINING")
print(f"    Model      : {MODEL_NAME}")
print(f"    Steps      : {total_steps}   ({TARGET_EPOCHS} epochs)")
print(f"    Optimizer  : adamw_bnb_8bit  |  GPU count : {torch.cuda.device_count()}")
print("═" * 65 + "\n")

try:
    trainer.train(resume_from_checkpoint=RESUME_FROM_CHECKPOINT)

    # ── Save Final Model ──────────────────────────────────────────────────────
    print(f"\n💾 Saving final model → {OUTPUT_PATH}")
    model.save_pretrained(OUTPUT_PATH)
    processor.save_pretrained(OUTPUT_PATH)
    print("✅ Training complete. Final model saved.")

    # ── Upload Final Model ────────────────────────────────────────────────────
    if UPLOAD_CHECKPOINTS and HF_TOKEN:
        upload_to_hf(
            OUTPUT_PATH, HF_OUTPUT_REPO,
            commit_msg=f"Final model — {TARGET_EPOCHS} epochs — {MODEL_NAME}",
        )
        print(f"✅ Final model uploaded to HuggingFace : {HF_OUTPUT_REPO}")

except Exception as e:
    import traceback
    print(f"\n❌ Training failed: {e}")
    traceback.print_exc()

finally:
    torch.cuda.empty_cache()
    gc.collect()
    print("\n🧹 GPU memory cleared.")